In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import re
import pandas as pd
from pathlib import Path
from terra_mystica_models.data import make_dataset

In [3]:
json_dir = Path(".").resolve().parent / "data" / "raw"

In [4]:
eg_json = json_dir / "2014-01.json"
with open(eg_json) as f:
    data = json.load(f)

In [5]:
eg_game = data[10]

In [6]:
tmg = make_dataset.TerraMysticaGame(eg_game)

In [7]:
tmg.game_name

'2p4'

In [8]:
tmg.game_date

datetime.datetime(2014, 1, 16, 22, 28, 53)

In [9]:
tmg.base_map

'126fe960806d587c78546b30f1a90853b1ada468'

In [10]:
tmg.players

{'witches': 'Duilen', 'mermaids': 'Lewdog'}

In [11]:
tmg.factions

['witches', 'mermaids']

In [12]:
tmg.faction_selection_order

{'mermaids': 1, 'witches': 2}

In [13]:
tmg.scoring_tile_order

{'SCORE5': 5, 'SCORE4': 2, 'SCORE7': 6, 'SCORE3': 4, 'SCORE8': 3, 'SCORE2': 1}

In [14]:
tmg.bonus_tiles

['BON6', 'BON4', 'BON9', 'BON3', 'BON5']

In [15]:
tmg.victory_points

{'witches': 117, 'mermaids': 114}

In [16]:
tmg.victory_points_margin

{'witches': 3, 'mermaids': 0}

In [17]:
tmg._check_player_count()

True

In [18]:
tmg._check_bonus_tile_count()

True

In [19]:
tmg._is_played_on_original_map()

True

In [20]:
tmg._has_nofaction_player()

False

To test my nofaction validation tool I'll have to find a game that actually has them

In [21]:
def find_nofaction_game():
    for json_file in json_dir.glob("*.json"):
        with open(json_file) as f:
            data = json.load(f) 
        for game in data:
            if any(faction["faction"].startswith("nofaction") for faction in game["factions"]):
                return game
nofaction_game = find_nofaction_game()

In [22]:
tmg2 = make_dataset.TerraMysticaGame(nofaction_game)

In [23]:
tmg2.factions

['nofaction1', 'witches', 'nomads']

In [24]:
tmg2.game_options

['option-strict-darkling-sh',
 'option-email-notify',
 'option-strict-chaosmagician-sh',
 'option-errata-cultist-power',
 'option-strict-leech']

In [25]:
tmg2._has_nofaction_player()

True

OK, next let's identify expansion games

In [26]:
def find_expansion_game():
    for json_file in json_dir.glob("*.json"):
        with open(json_file) as f:
            data = json.load(f) 
        for game in data:
            if any(faction["faction"] == "icemaidens" for faction in game["factions"]):
                return game

expansion_game = make_dataset.TerraMysticaGame(find_expansion_game())

In [27]:
expansion_game.factions

['darklings', 'icemaidens', 'witches', 'nomads']

In [28]:
expansion_game.game_options

['option-strict-darkling-sh',
 'option-email-notify',
 'option-strict-chaosmagician-sh',
 'option-mini-expansion-1',
 'option-errata-cultist-power',
 'option-strict-leech']

I don't see anything in options that would identify it, I'll probably just go off factions

In [29]:
expansion_game._is_played_on_original_map()

False

That might be something but I'll keep that logic separate

In [30]:
expansion_game._has_expansion_factions()

True

In [31]:
tmg2._has_expansion_factions()

False

In [32]:
tmg._has_expansion_factions()

False

Seems to be working

Ok, now let's identify if there are dropped players. Once that's done I think we're ready to build a dataset

In [33]:
def find_drop_faction_game():
    for json_file in json_dir.glob("*.json"):
        with open(json_file) as f:
            data = json.load(f) 
        for game in data:
            if any(key.startswith("drop") for key in game["events"]["global"]):
                return game

drop_game = make_dataset.TerraMysticaGame(find_drop_faction_game())

In [34]:
list(drop_game._raw_game["events"]["global"].keys())

['option-email-notify',
 'option-shipping-bonus',
 'SCORE1',
 'option-mini-expansion-1',
 'option-errata-cultist-power',
 'drop-faction',
 'faction-count',
 'SCORE5',
 'SCORE4',
 'option-strict-leech',
 'SCORE8',
 'SCORE3',
 'SCORE2']

In [35]:
drop_game._raw_game["events"]["global"]["drop-faction"]

{'round': {'all': 1, '5': 1}}

In [36]:
drop_game._has_dropped_faction()

True

In [37]:
tmg._has_dropped_faction()

False

Cool, coolcoolcool. Let's wrap all these tests into a nice dataset validator

In [38]:
tmg.is_valid_game

True

In [39]:
tmg2.is_valid_game

False

In [40]:
drop_game.is_valid_game

False

In [41]:
expansion_game.is_valid_game

False

# DataFrame Parsing

In [42]:
game_df = pd.DataFrame()
player_df = pd.DataFrame()

In [43]:
make_dataset._game_to_dfs(tmg, game_df, player_df)

True

In [44]:
game_df

,game_date,map,option-email-notify,option-shipping-bonus,option-errata-cultist-power,option-mini-expansion-1,option-strict-leech,faction1,faction2,tile_round_5,...,tile_round_1,BON6,BON4,BON9,BON3,BON5,witches_vp,mermaids_vp,witches_vp_margin,mermaids_vp_margin
2p4,2014-01-16 22:28:53,126fe960806d587c78546b30f1a90853b1ada468,True,True,True,True,True,mermaids,witches,SCORE5,...,SCORE2,True,True,True,True,True,117.0,114.0,3.0,0.0


In [45]:
player_df

,games,sum_vp,sum_vp_margin
Duilen,1.0,117.0,3.0
Lewdog,1.0,114.0,0.0


In [46]:
make_dataset._game_to_dfs(tmg2, game_df, player_df)

False

In [47]:
game_df

,game_date,map,option-email-notify,option-shipping-bonus,option-errata-cultist-power,option-mini-expansion-1,option-strict-leech,faction1,faction2,tile_round_5,...,tile_round_1,BON6,BON4,BON9,BON3,BON5,witches_vp,mermaids_vp,witches_vp_margin,mermaids_vp_margin
2p4,2014-01-16 22:28:53,126fe960806d587c78546b30f1a90853b1ada468,True,True,True,True,True,mermaids,witches,SCORE5,...,SCORE2,True,True,True,True,True,117.0,114.0,3.0,0.0


Sweeeeeeeet

# Let's fire it up

Have this here for troubleshooting, can delete after parsing works

In [ ]:
def find_error_game(game_name):
    for json_file in json_dir.glob("*.json"):
        with open(json_file) as f:
            data = json.load(f) 
        for game in data:
            if game_name == game["game"]:
                return game

error_game = make_dataset.TerraMysticaGame(find_error_game("10"))

In [49]:
game_df, player_df = make_dataset.games_to_dfs()

In [52]:
game_df

,game_date,map,faction1,faction2,faction3,tile_round_3,tile_round_6,tile_round_5,tile_round_1,tile_round_2,...,option-fire-and-ice-factions/variable,option-variable-turn-order,option-fire-and-ice-factions/variable_v2,option-fire-and-ice-factions/variable_v3,faction6,option-fire-and-ice-factions/variable_v4,faction7,option-temple-scoring-tile,option-fire-and-ice-factions/variable_v5,option-fire-and-ice-factions
pbc3,2013-02-10 02:40:12,126fe960806d587c78546b30f1a90853b1ada468,dwarves,darklings,halflings,SCORE5,SCORE4,SCORE7,SCORE1,SCORE8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,2013-03-29 18:14:03,126fe960806d587c78546b30f1a90853b1ada468,witches,chaosmagicians,swarmlings,SCORE7,SCORE4,SCORE5,SCORE8,SCORE1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21,2013-03-22 13:43:00,126fe960806d587c78546b30f1a90853b1ada468,nomads,mermaids,giants,SCORE6,SCORE7,SCORE8,SCORE1,SCORE5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2013-03-25 19:37:26,126fe960806d587c78546b30f1a90853b1ada468,nomads,alchemists,swarmlings,SCORE2,SCORE6,SCORE1,SCORE3,SCORE4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2013-03-03 00:42:28,126fe960806d587c78546b30f1a90853b1ada468,fakirs,swarmlings,chaosmagicians,SCORE2,SCORE7,SCORE4,SCORE6,SCORE1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZGame300,2020-01-17 17:01:59,126fe960806d587c78546b30f1a90853b1ada468,darklings,chaosmagicians,cultists,SCORE6,SCORE9,SCORE7,SCORE3,SCORE5,...,NaN,True,NaN,NaN,NaN,NaN,NaN,True,NaN,NaN
ZGame301,2020-01-14 01:17:45,126fe960806d587c78546b30f1a90853b1ada468,darklings,fakirs,dwarves,SCORE5,SCORE6,SCORE7,SCORE2,SCORE3,...,NaN,True,NaN,NaN,NaN,NaN,NaN,True,NaN,NaN
ZGame302,2020-01-20 00:07:50,126fe960806d587c78546b30f1a90853b1ada468,engineers,alchemists,cultists,SCORE2,SCORE7,SCORE6,SCORE5,SCORE8,...,NaN,True,NaN,NaN,NaN,NaN,NaN,True,NaN,NaN
ZGame303,2020-01-30 17:12:49,126fe960806d587c78546b30f1a90853b1ada468,swarmlings,nomads,chaosmagicians,SCORE1,SCORE7,SCORE6,SCORE8,SCORE9,...,NaN,True,NaN,NaN,NaN,NaN,NaN,True,NaN,NaN


In [53]:
player_df

,games,sum_vp,sum_vp_margin
NaN,283.0,31880.0,5050.0
jsnell,231.0,31525.0,6730.0
dkeisen,11.0,1376.0,391.0
Jan,259.0,33705.0,5083.0
Lachlan,15.0,1797.0,411.0
...,...,...,...
Mikeyloo,2.0,107.0,35.0
boardinthepark,1.0,33.0,0.0
toby776,1.0,56.0,0.0
iremina,4.0,290.0,0.0
